In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        continue

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Loading and Analysis of DataFrame

In [ ]:
import os
import csv

def categorize_image(directory):
    if directory.startswith('S_') or directory.endswith('Bad') or directory.startswith('Rotten') or directory=='Old' or directory=='Dried' or directory=='Damaged' or directory=='Formalin-mixed' in directory:
        return 'Rotten'
    else:
        return 'Fresh'

def create_csv(root_dir, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['Image Directory', 'Main Directory', 'Category', 'Sub Category', 'Sub Sub Category']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for main_dir in ['Fruits', 'Vegetables']:
            main_path = os.path.join(root_dir, 'Detection', main_dir)
            
            for category in os.listdir(main_path):
                category_path = os.path.join(main_path, category)
                
                if os.path.isdir(category_path):
                    for sub_category in os.listdir(category_path):
                        sub_category_path = os.path.join(category_path, sub_category)
                        
                        if os.path.isdir(sub_category_path):
                            sub_sub_category = categorize_image(sub_category)
                            
                            for image in os.listdir(sub_category_path):
                                if image.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                                    image_path = os.path.join(sub_category_path, image)
                                    writer.writerow({
                                        'Image Directory': image_path,
                                        'Main Directory': main_dir,
                                        'Category': category,
                                        'Sub Category': sub_category,
                                        'Sub Sub Category': sub_sub_category
                                    })

# Usage
root_directory = '/kaggle/input/woking-dataset'
output_csv = 'fruit_vegetable_dataset.csv'
create_csv(root_directory, output_csv)

In [ ]:
import pandas as pd
df=pd.read_csv(output_csv)

In [ ]:
df

In [ ]:
df[df['Category']=='1. Bell Pepper']

In [ ]:
df[df['Category']=='Spinach']

In [ ]:
df[df['Sub Sub Category']=='Fresh']

In [ ]:
df[df['Sub Sub Category']!='Fresh']

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

# Tried Multiple things (no need to run)

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from tqdm import tqdm

# Ensure GPU usage if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the dataset
df = pd.read_csv('fruit_vegetable_dataset.csv')

# Map categories to numerical labels
categories = df['Category'].unique()
category_to_idx = {category: idx for idx, category in enumerate(categories)}
df['Label'] = df['Category'].map(category_to_idx)

# Split the dataset into train, validation, and test sets
train_val, test_df = train_test_split(df, test_size=0.2, stratify=df['Label'])
train_df, val_df = train_test_split(train_val, test_size=0.2, stratify=train_val['Label'])

# Custom Dataset class for loading images and labels
class FruitVegDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['Image Directory']
        image = Image.open(img_path).convert('RGB')
        label = self.dataframe.iloc[idx]['Label']

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations for the training and validation sets
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_val_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets and dataloaders
train_dataset = FruitVegDataset(train_df, transform=transform_train)
val_dataset = FruitVegDataset(val_df, transform=transform_val_test)
test_dataset = FruitVegDataset(test_df, transform=transform_val_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Function to train and evaluate a model using ResNet18 (similar approach can be applied to ResNet50)
def train_and_evaluate_resnet():
    # Load pre-trained ResNet18 model and modify the final layer for our dataset
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, len(categories))
    
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    num_epochs = 10
    best_accuracy = 0

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        
        for images, labels in tqdm(train_loader, desc=f'Training ResNet18 Epoch {epoch+1}/{num_epochs}'):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)

        # Validation phase
        model.eval()
        corrects = 0
        
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f'Validating ResNet18'):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                corrects += torch.sum(preds == labels).item()

        epoch_accuracy = corrects / len(val_loader.dataset)

        print(f'ResNet18 - Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

        # Save the best model based on validation accuracy
        if epoch_accuracy > best_accuracy:
            best_accuracy = epoch_accuracy
            best_model_wts = model.state_dict()

    # Load best model weights and evaluate on test set
    model.load_state_dict(best_model_wts)
    
    # Test phase
    model.eval()
    test_corrects = 0
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc=f'Testing ResNet18'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            test_corrects += torch.sum(preds == labels).item()

    test_accuracy = test_corrects / len(test_loader.dataset)
    print(f'ResNet18 - Test Accuracy: {test_accuracy:.4f}')

    # Save the trained model
    torch.save(model.state_dict(), 'resnet18_fruit_veg.pth')
    print("Model saved as 'resnet18_fruit_veg.pth'")

# Train and evaluate the ResNet18 model
train_and_evaluate_resnet()

In [ ]:
import torch,gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import ViTForImageClassification, ViTImageProcessor, TrainingArguments, Trainer
from transformers import DefaultDataCollator
import torch
from PIL import Image
import gc

# Ensure GPU usage if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the dataset
#df = pd.read_csv('/kaggle/input/your-dataset-name/fruit_vegetable_dataset.csv')

# Map categories to numerical labels
categories = df['Category'].unique()
category_to_idx = {category: idx for idx, category in enumerate(categories)}
df['Label'] = df['Category'].map(category_to_idx)

# Split the dataset into train, validation, and test sets
train_val_df, test_df = train_test_split(df, test_size=0.2, stratify=df['Label'])
train_df, val_df = train_test_split(train_val_df, test_size=0.2, stratify=train_val_df['Label'])

# Convert DataFrame to Hugging Face Dataset format
def df_to_dataset(dataframe):
    return Dataset.from_pandas(dataframe[['Image Directory', 'Label']])

train_dataset = df_to_dataset(train_df)
val_dataset = df_to_dataset(val_df)
test_dataset = df_to_dataset(test_df)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Load ViT Image Processor
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224', device=device)

# Define a function to preprocess images and manage memory
def preprocess_images(examples):
    images = []
    for image_path in examples['Image Directory']:
        image = Image.open(image_path).convert("RGB")
        images.append(image)
    
    # Apply processing and convert to PyTorch tensors
    processed_images = processor(images=images, return_tensors="pt")

    # Clear memory after each batch
    del images
    gc.collect()
    torch.cuda.empty_cache()
    
    # Return processed images and labels
    return {
        'pixel_values': processed_images['pixel_values'],
        'labels': examples['Label']
    }

# Apply preprocessing to datasets using batched mapping for efficiency
dataset_dict = dataset_dict.map(preprocess_images, batched=True, batch_size=32)

# Load pre-trained ViT model for classification
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=len(categories),
    id2label={i: label for i, label in enumerate(categories)},
    label2id={label: i for i, label in enumerate(categories)}
).to(device)

# Set training arguments, including smaller batch size to manage memory usage
training_args = TrainingArguments(
    output_dir="./vit-model",
    per_device_train_batch_size=4,  # Reduce batch size for lower memory consumption
    per_device_eval_batch_size=4,   # Reduce eval batch size as well
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True
)

# Define data collator
data_collator = DefaultDataCollator()

# Define the metrics computation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    accuracy = (predictions == labels).float().mean()
    return {"accuracy": accuracy.item()}

# Initialize Trainer with memory management considerations
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['validation'],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start training with GPU memory management in place
trainer.train()

# Evaluate the model on the test dataset with memory management
metrics = trainer.evaluate(dataset_dict['test'])
print(metrics)

# Save the trained model
trainer.save_model("vit_fruit_veg_classifier")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import ViTForImageClassification, ViTImageProcessor, TrainingArguments, Trainer
from transformers import DefaultDataCollator
import torch
from PIL import Image
import gc

# Ensure GPU usage if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the dataset
df = pd.read_csv('fruit_vegetable_dataset.csv')

# Map categories to numerical labels
categories = df['Category'].unique()
category_to_idx = {category: idx for idx, category in enumerate(categories)}
df['Label'] = df['Category'].map(category_to_idx)

# Split the dataset into train, validation, and test sets
train_val_df, test_df = train_test_split(df, test_size=0.2, stratify=df['Label'])
train_df, val_df = train_test_split(train_val_df, test_size=0.2, stratify=train_val_df['Label'])

# Convert DataFrame to Hugging Face Dataset format
def df_to_dataset(dataframe):
    return Dataset.from_pandas(dataframe[['Image Directory', 'Label']])

train_dataset = df_to_dataset(train_df)
val_dataset = df_to_dataset(val_df)
test_dataset = df_to_dataset(test_df)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Load ViT Image Processor
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')

# Define a function to preprocess images in chunks and manage memory
def preprocess_images_in_chunks(dataset, chunk_size=1000):
    processed_batches = []
    for i in range(0, len(dataset), chunk_size):
        batch = dataset[i:i+chunk_size]
        images = [Image.open(image_path).convert("RGB") for image_path in batch['Image Directory']]
        processed_images = processor(images=images, return_tensors="pt")
        
        # Dispose of images and clear memory after processing each chunk
        del images
        gc.collect()
        torch.cuda.empty_cache()
        
        processed_batches.append(processed_images)
    
    return processed_batches

# Preprocess datasets in chunks to handle large data efficiently
train_processed_batches = preprocess_images_in_chunks(dataset_dict['train'])
val_processed_batches = preprocess_images_in_chunks(dataset_dict['validation'])
test_processed_batches = preprocess_images_in_chunks(dataset_dict['test'])

# Load pre-trained ViT model and modify for our task
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=len(categories),
    id2label={i: label for i, label in enumerate(categories)},
    label2id={label: i for i, label in enumerate(categories)}
).to(device)

# Define training arguments with lower batch size if needed to fit into memory
training_args = TrainingArguments(
    output_dir="./vit-model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
)

# Define data collator
data_collator = DefaultDataCollator()

# Define metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    accuracy = (predictions == labels).float().mean()
    return {"accuracy": accuracy.item()}

# Initialize Trainer with memory management considerations
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_processed_batches,
    eval_dataset=val_processed_batches,
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train and evaluate the model with memory management steps included in the training loop
trainer.train()

# Evaluate on the test set with memory management considerations
metrics = trainer.evaluate(test_processed_batches)
print(metrics)

# Save the trained model with consideration for memory management
trainer.save_model("vit_fruit_veg_classifier")

# Data Category , Sub Category and Sub Sub Category Analysis 

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('fruit_vegetable_dataset.csv')

# Display the count of each category
category_counts = df['Category'].value_counts()
print("Category Counts:")
print(category_counts)

# Optionally, display counts for subcategories and sub-subcategories as well
sub_category_counts = df['Sub Category'].value_counts()
print("\nSub Category Counts:")
print(sub_category_counts)

sub_sub_category_counts = df['Sub Sub Category'].value_counts()
print("\nSub Sub Category Counts:")
print(sub_sub_category_counts)

# For downloading it to System and also tried annotating and Equalizing data (Tried but not used) (No need to run)

In [ ]:
import pandas as pd
from sklearn.utils import resample
from torchvision import transforms
from PIL import Image
import os

# Load the dataset
df = pd.read_csv('fruit_vegetable_dataset.csv')

# Define the target number of images per category
TARGET_IMAGES_PER_CATEGORY = 3000

# Define image augmentation transformations for oversampling
augmentation_transforms = transforms.Compose([
    transforms.RandomRotation(degrees=15),
    transforms.RandomHorizontalFlip(),
])

def augment_image(image_path, output_dir, augment_count):
    """Augment and save images."""
    image = Image.open(image_path).convert('RGB')
    for i in range(augment_count):
        augmented_image = augmentation_transforms(image)
        base_name = os.path.basename(image_path)
        name, ext = os.path.splitext(base_name)
        augmented_image_path = os.path.join(output_dir, f"{name}_aug_{i}{ext}")
        augmented_image.save(augmented_image_path)
        yield augmented_image_path

def balance_dataset(df):
    balanced_data = []

    # Create a directory for augmented images if it doesn't exist
    aug_dir = 'augmented_images'
    os.makedirs(aug_dir, exist_ok=True)

    # Group by Category, Sub Category, and Sub Sub Category
    for (category, sub_category, sub_sub_category), group in df.groupby(['Category', 'Sub Category', 'Sub Sub Category']):
        current_count = len(group)
        
        if current_count < TARGET_IMAGES_PER_CATEGORY:
            # Oversample by augmenting images
            needed_images = TARGET_IMAGES_PER_CATEGORY - current_count
            augment_per_image = needed_images // current_count + 1
            
            for _, row in group.iterrows():
                image_path = row['Image Directory']
                augmented_paths = list(augment_image(image_path, aug_dir, augment_per_image))
                augmented_rows = [row.copy() for _ in augmented_paths]
                
                for aug_row, aug_path in zip(augmented_rows, augmented_paths):
                    aug_row['Image Directory'] = aug_path
                
                balanced_data.extend(augmented_rows)
            
            # Add original data to balanced data
            balanced_data.extend(group.to_dict('records'))
        
        elif current_count > TARGET_IMAGES_PER_CATEGORY:
            # Undersample by taking a random sample of the group
            sampled_group = group.sample(n=TARGET_IMAGES_PER_CATEGORY, random_state=42)
            balanced_data.extend(sampled_group.to_dict('records'))
        
        else:
            # If already at target size, add directly
            balanced_data.extend(group.to_dict('records'))
    
    return pd.DataFrame(balanced_data)

# Balance the dataset
balanced_df = balance_dataset(df)

# Check the distribution after balancing
print(balanced_df['Category'].value_counts())
print(balanced_df['Sub Category'].value_counts())
print(balanced_df['Sub Sub Category'].value_counts())

# Save the balanced dataset to a new CSV file (optional)
balanced_df.to_csv('balanced_fruit_vegetable_dataset.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.utils import resample
import albumentations as A
import cv2
import os
from tqdm import tqdm
import shutil

# Load the dataset
df = pd.read_csv('fruit_vegetable_dataset.csv')

# Define the target number of images per category
TARGET_IMAGES_PER_CATEGORY = 3000

# Define image augmentation transformations using Albumentations
augmentation_transforms = A.Compose([
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
])

def augment_image(image_path, output_dir, augment_count):
    """Augment and save images."""
    image = cv2.imread(image_path)
    for i in range(augment_count):
        augmented = augmentation_transforms(image=image)
        augmented_image = augmented['image']
        base_name = os.path.basename(image_path)
        name, ext = os.path.splitext(base_name)
        augmented_image_path = os.path.join(output_dir, f"{name}_aug_{i}{ext}")
        cv2.imwrite(augmented_image_path, augmented_image)
        yield augmented_image_path

def balance_dataset(df):
    balanced_data = []

    # Create a directory for augmented images if it doesn't exist
    aug_dir = 'augmented_images'
    os.makedirs(aug_dir, exist_ok=True)

    # Group by Category, Sub Category, and Sub Sub Category
    for (category, sub_category, sub_sub_category), group in df.groupby(['Category', 'Sub Category', 'Sub Sub Category']):
        current_count = len(group)
        
        if current_count < TARGET_IMAGES_PER_CATEGORY:
            # Oversample by augmenting images
            needed_images = TARGET_IMAGES_PER_CATEGORY - current_count
            augment_per_image = needed_images // current_count + 1
            
            for _, row in group.iterrows():
                image_path = row['Image Directory']
                augmented_paths = list(augment_image(image_path, aug_dir, augment_per_image))
                augmented_rows = [row.copy() for _ in augmented_paths]
                
                for aug_row, aug_path in zip(augmented_rows, augmented_paths):
                    aug_row['Image Directory'] = aug_path
                
                balanced_data.extend(augmented_rows)
            
            # Add original data to balanced data
            balanced_data.extend(group.to_dict('records'))
        
        elif current_count > TARGET_IMAGES_PER_CATEGORY:
            # Undersample by taking a random sample of the group
            sampled_group = group.sample(n=TARGET_IMAGES_PER_CATEGORY, random_state=42)
            balanced_data.extend(sampled_group.to_dict('records'))
        
        else:
            # If already at target size, add directly
            balanced_data.extend(group.to_dict('records'))
    
    return pd.DataFrame(balanced_data)

# Balance the dataset
balanced_df = balance_dataset(df)

# Check the distribution after balancing
print(balanced_df['Category'].value_counts())
print(balanced_df['Sub Category'].value_counts())
print(balanced_df['Sub Sub Category'].value_counts())

# Save the balanced dataset to a new CSV file (optional)
balanced_df.to_csv('balanced_fruit_vegetable_dataset.csv', index=False)

# Create a zip file of the balanced dataset images
zip_dir = 'balanced_dataset_zip'
os.makedirs(zip_dir, exist_ok=True)

for _, row in tqdm(balanced_df.iterrows(), total=len(balanced_df)):
    src_path = row['Image Directory']
    dst_path = os.path.join(zip_dir, os.path.basename(src_path))
    shutil.copy(src_path, dst_path)

shutil.make_archive('balanced_fruit_vegetable_dataset', 'zip', zip_dir)

print("Zipped dataset created: balanced_fruit_vegetable_dataset.zip")

In [ ]:
import pandas as pd
import albumentations as A
import cv2
import os
from tqdm import tqdm

# Load the dataset
df = pd.read_csv('fruit_vegetable_dataset.csv')

# Define the target number of images per category
TARGET_IMAGES_PER_CATEGORY = 3000

# Define image augmentation transformations using Albumentations
augmentation_transforms = A.Compose([
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
])

def augment_image(image_path, output_dir, augment_count):
    """Augment and save images."""
    image = cv2.imread(image_path)
    for i in range(augment_count):
        augmented = augmentation_transforms(image=image)
        augmented_image = augmented['image']
        base_name = os.path.basename(image_path)
        name, ext = os.path.splitext(base_name)
        augmented_image_path = os.path.join(output_dir, f"{name}_aug_{i}{ext}")
        cv2.imwrite(augmented_image_path, augmented_image)
        yield augmented_image_path

def balance_dataset(df):
    balanced_data = []

    # Create a directory for augmented images if it doesn't exist
    aug_dir = 'augmented_images'
    os.makedirs(aug_dir, exist_ok=True)

    # Group by Category and Sub Category
    for (category, sub_category), group in df.groupby(['Category', 'Sub Category']):
        current_count = len(group)
        
        print(f"Processing {category} - {sub_category}: {current_count} images")

        if current_count < TARGET_IMAGES_PER_CATEGORY:
            # Oversample by augmenting images
            needed_images = TARGET_IMAGES_PER_CATEGORY - current_count
            augment_per_image = needed_images // current_count + 1
            
            for _, row in tqdm(group.iterrows(), total=current_count, desc=f"Augmenting {category} - {sub_category}"):
                image_path = row['Image Directory']
                augmented_paths = list(augment_image(image_path, aug_dir, augment_per_image))
                augmented_rows = [row.copy() for _ in augmented_paths]
                
                for aug_row, aug_path in zip(augmented_rows, augmented_paths):
                    aug_row['Image Directory'] = aug_path
                
                balanced_data.extend(augmented_rows)
            
            # Add original data to balanced data
            balanced_data.extend(group.to_dict('records'))
        
        elif current_count > TARGET_IMAGES_PER_CATEGORY:
            # Undersample by taking a random sample of the group
            sampled_group = group.sample(n=TARGET_IMAGES_PER_CATEGORY, random_state=42)
            balanced_data.extend(sampled_group.to_dict('records'))
        
        else:
            # If already at target size, add directly
            balanced_data.extend(group.to_dict('records'))
    
    return pd.DataFrame(balanced_data)

# Balance the dataset
balanced_df = balance_dataset(df)

# Check the distribution after balancing
print(balanced_df['Category'].value_counts())
print(balanced_df['Sub Category'].value_counts())

# Save the balanced dataset to a new CSV file
balanced_df.to_csv('balanced_fruit_vegetable_dataset.csv', index=False)

# Organize images into a structured directory (optional)
output_dir = 'balanced_dataset_images'
os.makedirs(output_dir, exist_ok=True)

for _, row in balanced_df.iterrows():
    category_dir = os.path.join(output_dir, row['Category'])
    os.makedirs(category_dir, exist_ok=True)
    
    src_path = row['Image Directory']
    dst_path = os.path.join(category_dir, os.path.basename(src_path))
    
    if not os.path.exists(dst_path):
        os.rename(src_path, dst_path)

In [ ]:
import pandas as pd
import albumentations as A
import cv2
import os
from tqdm import tqdm
from sklearn.utils import resample
import shutil
import zipfile

# Load the dataset
df = pd.read_csv('fruit_vegetable_dataset.csv')

# Define the target number of images per category
TARGET_IMAGES_PER_CATEGORY = 3000

# Define image augmentation transformations using Albumentations
augmentation_transforms = A.Compose([
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
])

def augment_image(image_path, output_dir, augment_count):
    """Augment and save images."""
    image = cv2.imread(image_path)
    for i in range(augment_count):
        augmented = augmentation_transforms(image=image)
        augmented_image = augmented['image']
        base_name = os.path.basename(image_path)
        name, ext = os.path.splitext(base_name)
        augmented_image_path = os.path.join(output_dir, f"{name}_aug_{i}{ext}")
        cv2.imwrite(augmented_image_path, augmented_image)
        yield augmented_image_path

def balance_dataset(df):
    balanced_data = []

    # Create a directory for the balanced dataset
    balanced_dir = 'balanced_dataset'
    os.makedirs(balanced_dir, exist_ok=True)

    # Group by Category, Sub Category, and Sub Sub Category
    for (category, sub_category, sub_sub_category), group in df.groupby(['Category', 'Sub Category', 'Sub Sub Category']):
        current_count = len(group)
        
        # Create subdirectory for each category/subcategory/sub-subcategory
        category_dir = os.path.join(balanced_dir, category, sub_category, sub_sub_category)
        os.makedirs(category_dir, exist_ok=True)

        if current_count < TARGET_IMAGES_PER_CATEGORY:
            # Oversample by augmenting images
            needed_images = TARGET_IMAGES_PER_CATEGORY - current_count
            augment_per_image = needed_images // current_count + 1
            
            for _, row in group.iterrows():
                image_path = row['Image Directory']
                augmented_paths = list(augment_image(image_path, category_dir, augment_per_image))
                augmented_rows = [row.copy() for _ in augmented_paths]
                
                for aug_row, aug_path in zip(augmented_rows, augmented_paths):
                    aug_row['Image Directory'] = aug_path
                
                balanced_data.extend(augmented_rows)
            
            # Add original data to balanced data
            balanced_data.extend(group.to_dict('records'))
        
        elif current_count > TARGET_IMAGES_PER_CATEGORY:
            # Undersample by taking a random sample of the group
            sampled_group = group.sample(n=TARGET_IMAGES_PER_CATEGORY, random_state=42)
            for _, row in sampled_group.iterrows():
                image_path = row['Image Directory']
                shutil.copy(image_path, category_dir)
            balanced_data.extend(sampled_group.to_dict('records'))
        
        else:
            # If already at target size, add directly and copy files
            for _, row in group.iterrows():
                image_path = row['Image Directory']
                shutil.copy(image_path, category_dir)
            balanced_data.extend(group.to_dict('records'))
    
    return pd.DataFrame(balanced_data)

# Balance the dataset
balanced_df = balance_dataset(df)

# Check the distribution after balancing
print(balanced_df['Category'].value_counts())
print(balanced_df['Sub Category'].value_counts())
print(balanced_df['Sub Sub Category'].value_counts())

# Save the balanced dataset to a new CSV file
balanced_csv_path = 'balanced_fruit_vegetable_dataset.csv'
balanced_df.to_csv(balanced_csv_path, index=False)

# Zip the balanced dataset directory
zip_filename = 'balanced_dataset.zip'
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(balanced_dir):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, balanced_dir))

print(f"Balanced dataset and CSV saved. Zip file created: {zip_filename}")

In [ ]:
import pandas as pd
import albumentations as A
import cv2
import os
from tqdm import tqdm
import shutil
import zipfile

# Load the dataset
df = pd.read_csv('fruit_vegetable_dataset.csv')

# Define the target number of images per category
TARGET_IMAGES_PER_CATEGORY = 3000

# Define image augmentation transformations using Albumentations
augmentation_transforms = A.Compose([
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
])

def augment_image(image_path, output_dir, augment_count):
    """Augment and save images."""
    image = cv2.imread(image_path)
    for i in range(augment_count):
        augmented = augmentation_transforms(image=image)
        augmented_image = augmented['image']
        base_name = os.path.basename(image_path)
        name, ext = os.path.splitext(base_name)
        augmented_image_path = os.path.join(output_dir, f"{name}_aug_{i}{ext}")
        cv2.imwrite(augmented_image_path, augmented_image)
        yield augmented_image_path

def balance_dataset(df):
    balanced_data = []

    # Create a directory for the balanced dataset
    balanced_dir = 'balanced_dataset'
    os.makedirs(balanced_dir, exist_ok=True)

    # Group by Category and Sub Category
    for category, group in df.groupby('Category'):
        current_count = len(group)
        
        # Create subdirectory for each category
        category_dir = os.path.join(balanced_dir, category)
        os.makedirs(category_dir, exist_ok=True)

        if current_count < TARGET_IMAGES_PER_CATEGORY:
            # Oversample by augmenting images
            needed_images = TARGET_IMAGES_PER_CATEGORY - current_count
            augment_per_image = needed_images // current_count + 1
            
            for _, row in group.iterrows():
                image_path = row['Image Directory']
                augmented_paths = list(augment_image(image_path, category_dir, augment_per_image))
                augmented_rows = [row.copy() for _ in augmented_paths]
                
                for aug_row, aug_path in zip(augmented_rows, augmented_paths):
                    aug_row['Image Directory'] = aug_path
                
                balanced_data.extend(augmented_rows)
            
            # Add original data to balanced data
            balanced_data.extend(group.to_dict('records'))
        
        elif current_count > TARGET_IMAGES_PER_CATEGORY:
            # Undersample by taking a random sample of the group
            sampled_group = group.sample(n=TARGET_IMAGES_PER_CATEGORY, random_state=42)
            for _, row in sampled_group.iterrows():
                image_path = row['Image Directory']
                shutil.copy(image_path, category_dir)
            balanced_data.extend(sampled_group.to_dict('records'))
        
        else:
            # If already at target size, add directly and copy files
            for _, row in group.iterrows():
                image_path = row['Image Directory']
                shutil.copy(image_path, category_dir)
            balanced_data.extend(group.to_dict('records'))
    
    return pd.DataFrame(balanced_data)

# Balance the dataset
balanced_df = balance_dataset(df)

# Check the distribution after balancing
print(balanced_df['Category'].value_counts())

# Save the balanced dataset to a new CSV file
balanced_csv_path = 'balanced_fruit_vegetable_dataset.csv'
balanced_df.to_csv(balanced_csv_path, index=False)

# Zip the balanced dataset directory
zip_filename = 'balanced_dataset.zip'
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk('balanced_dataset'):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, 'balanced_dataset'))

print(f"Balanced dataset and CSV saved. Zip file created: {zip_filename}")

In [ ]:
import pandas as pd
import numpy as np

# Sample DataFrame (replace this with your actual DataFrame)

# Function to create the balanced DataFrame
def create_balanced_df(df, target_count=3000):
    balanced_df = []

    for category, group in df.groupby('Category'):
        subcategory_groups = group.groupby('Sub Category')
        num_subcategories = len(subcategory_groups)

        # Calculate how many images to take from each subcategory
        images_per_subcategory = target_count // num_subcategories

        for sub_category, sub_group in subcategory_groups:
            current_count = len(sub_group)

            if current_count < images_per_subcategory:
                # Duplicate the group until we reach the required count
                duplicates_needed = images_per_subcategory // current_count
                remainder_needed = images_per_subcategory % current_count
                
                # Create the balanced group
                balanced_group = pd.concat(
                    [sub_group] * duplicates_needed +
                    [sub_group.sample(remainder_needed, replace=True)]
                )
            else:
                # Sample images if we have more than needed
                balanced_group = sub_group.sample(images_per_subcategory, random_state=1)

            balanced_df.append(balanced_group)

    return pd.concat(balanced_df, ignore_index=True)

# Create the new balanced DataFrame
new_df = create_balanced_df(df, target_count=3000)

# Output the new DataFrame
print(new_df)
print(f"New DataFrame shape: {new_df.shape}")


In [ ]:
import pandas as pd


# Display the count of each category
category_counts = new_df['Category'].value_counts()
print("Category Counts:")
print(category_counts)

# Optionally, display counts for subcategories and sub-subcategories as well
sub_category_counts = new_df['Sub Category'].value_counts()
print("\nSub Category Counts:")
print(sub_category_counts)

sub_sub_category_counts = new_df['Sub Sub Category'].value_counts()
print("\nSub Sub Category Counts:")
print(sub_sub_category_counts)

# Efficient Net For Category 

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from tqdm import tqdm

# Ensure GPU usage if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Map categories to numerical labels
categories = df['Category'].unique()
category_to_idx = {category: idx for idx, category in enumerate(categories)}
df['Label'] = df['Category'].map(category_to_idx)

# Split the dataset into train, validation, and test sets
train_val_df, test_df = train_test_split(df, test_size=0.2, stratify=df['Label'])
train_df, val_df = train_test_split(train_val_df, test_size=0.2, stratify=train_val_df['Label'])

# Custom Dataset class for loading images and labels
class FruitVegDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['Image Directory']
        image = Image.open(img_path).convert('RGB')
        label = self.dataframe.iloc[idx]['Label']

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations for the training and validation sets
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_val_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets and dataloaders
train_dataset = FruitVegDataset(train_df, transform=transform_train)
val_dataset = FruitVegDataset(val_df, transform=transform_val_test)
test_dataset = FruitVegDataset(test_df, transform=transform_val_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

# Load pre-trained EfficientNet B7 model and modify the final layer for our dataset
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(categories))
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Function to train and evaluate the model
def train_and_evaluate():
    num_epochs = 1
    best_accuracy = 0

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        
        for images, labels in tqdm(train_loader, desc=f'Training Epoch {epoch+1}/{num_epochs}'):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)
            #print(f'Running loss {running_loss}')

        epoch_loss = running_loss / len(train_loader.dataset)

        # Validation phase
        model.eval()
        corrects = 0
        
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f'Validating'):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                corrects += torch.sum(preds == labels).item()

        epoch_accuracy = corrects / len(val_loader.dataset)

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

        # Save the best model based on validation accuracy
        if epoch_accuracy > best_accuracy:
            best_accuracy = epoch_accuracy
            best_model_wts = model.state_dict()

    # Load best model weights and evaluate on test set
    model.load_state_dict(best_model_wts)
    
    # Test phase
    model.eval()
    test_corrects = 0
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc=f'Testing'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            test_corrects += torch.sum(preds == labels).item()

    test_accuracy = test_corrects / len(test_loader.dataset)
    print(f'Test Accuracy: {test_accuracy:.4f}')

    # Save the trained model
    torch.save(model.state_dict(), 'efficientnet_b0_fruit_veg-w.pth')
    print("Model saved as 'efficientnet_b0_fruit_veg_2.pth")

# Train and evaluate the EfficientNet B7 model
train_and_evaluate()

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import pandas as pd

# Ensure GPU usage if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

categories = df['Category'].unique()
category_to_idx = {category: idx for idx, category in enumerate(categories)}
idx_to_category = {idx: category for category, idx in category_to_idx.items()}

# Load the saved model
model = models.efficientnet_b0(pretrained=False)  # Load without pre-trained weights
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(categories))
model.load_state_dict(torch.load('/kaggle/working/efficientnet_b0_fruit_veg-w.pth', map_location=device))
model.to(device)
model.eval()

# Define the transformation for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def predict_image_class(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, predicted_label = torch.max(outputs, 1)

    predicted_label = predicted_label.item()
    predicted_category = idx_to_category[predicted_label]

    return predicted_label, predicted_category


# Example usage:
image_path = '/kaggle/input/testing-of-me/RG.jpg'  # Replace with the path to your uploaded image
predicted_label, predicted_category = predict_image_class(image_path)

print("Predicted label:", predicted_label)
print("Predicted category:", predicted_category)

# Display the labelled classes and their actual names
print("\nLabelled classes and their actual names:")
for idx, category in idx_to_category.items():
    print(f"Label {idx}: {category}")

# Efficient Net For Sub Sub Category of Freshness

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from tqdm import tqdm

# Ensure GPU usage if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Map categories to numerical labels
categories = df['Sub Sub Category'].unique()
category_to_idx = {category: idx for idx, category in enumerate(categories)}
df['Label'] = df['Sub Sub Category'].map(category_to_idx)

# Split the dataset into train, validation, and test sets
train_val_df, test_df = train_test_split(df, test_size=0.2, stratify=df['Label'])
train_df, val_df = train_test_split(train_val_df, test_size=0.2, stratify=train_val_df['Label'])

# Custom Dataset class for loading images and labels
class FruitVegDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['Image Directory']
        image = Image.open(img_path).convert('RGB')
        label = self.dataframe.iloc[idx]['Label']

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations for the training and validation sets
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_val_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets and dataloaders
train_dataset = FruitVegDataset(train_df, transform=transform_train)
val_dataset = FruitVegDataset(val_df, transform=transform_val_test)
test_dataset = FruitVegDataset(test_df, transform=transform_val_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

# Load pre-trained EfficientNet B7 model and modify the final layer for our dataset
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(categories))
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Function to train and evaluate the model
def train_and_evaluate():
    num_epochs = 2
    best_accuracy = 0

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        
        for images, labels in tqdm(train_loader, desc=f'Training Epoch {epoch+1}/{num_epochs}'):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)
            #print(f'Running loss {running_loss}')

        epoch_loss = running_loss / len(train_loader.dataset)

        # Validation phase
        model.eval()
        corrects = 0
        
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f'Validating'):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                corrects += torch.sum(preds == labels).item()

        epoch_accuracy = corrects / len(val_loader.dataset)

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}')

        # Save the best model based on validation accuracy
        if epoch_accuracy > best_accuracy:
            best_accuracy = epoch_accuracy
            best_model_wts = model.state_dict()

    # Load best model weights and evaluate on test set
    model.load_state_dict(best_model_wts)
    
    # Test phase
    model.eval()
    test_corrects = 0
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc=f'Testing'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            test_corrects += torch.sum(preds == labels).item()

    test_accuracy = test_corrects / len(test_loader.dataset)
    print(f'Test Accuracy: {test_accuracy:.4f}')

    # Save the trained model
    torch.save(model.state_dict(), 'efficientnet_b0_fruit_veg.pth')
    print("Model saved as 'efficientnet_b0_fruit_veg.pth'")

# Train and evaluate the EfficientNet B7 model
train_and_evaluate()

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import pandas as pd

# Ensure GPU usage if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

categories = df['Sub Sub Category'].unique()
category_to_idx = {category: idx for idx, category in enumerate(categories)}
idx_to_category = {idx: category for category, idx in category_to_idx.items()}

# Load the saved model
model = models.efficientnet_b0(pretrained=False)  # Load without pre-trained weights
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(categories))
model.load_state_dict(torch.load('/content/efficientnet_b0_fruit_veg.pth', map_location=device))
model.to(device)
model.eval()

# Define the transformation for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def predict_image_class(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, predicted_label = torch.max(outputs, 1)

    predicted_label = predicted_label.item()
    predicted_category = idx_to_category[predicted_label]

    return predicted_label, predicted_category


# Example usage:
image_path = '/content/M.jpg'  # Replace with the path to your uploaded image
predicted_label, predicted_category = predict_image_class(image_path)

print("Predicted label:", predicted_label)
print("Predicted category:", predicted_category)

# Display the labelled classes and their actual names
print("\nLabelled classes and their actual names:")
for idx, category in idx_to_category.items():
    print(f"Label {idx}: {category}")